In [ ]:
!pip install opendatasets --upgrade
!pip install isodate

In [1]:
import numpy as np
import json
import requests
import heapq
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from io import BytesIO
import pickle
import pandas as pd
import time
import opendatasets as od
import isodate
# from tqdm.notebook import tqdm

from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import torch

c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
od.download_kaggle_dataset("https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews", "data")

Skipping, found downloaded files in "data\foodcom-recipes-and-reviews" (use force=True to force download)


# Embedding de las instrucciones

In [ ]:
recipes = pd.read_parquet("data/foodcom-recipes-and-reviews/recipes.parquet")
recipes.head()

In [ ]:
recipes

In [ ]:
recipes['RecipeInstructions'] = recipes['RecipeInstructions'].apply(lambda steps: ' '.join(steps))

In [ ]:
recipes['RecipeInstructions'].head()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

In [ ]:
embeddings = []
for instruction in tqdm(recipes['RecipeInstructions']):
    inputs = tokenizer(instruction, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    instruction_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    embeddings.append(instruction_embedding)

In [ ]:
embeddings_matrix = np.vstack(embeddings)

In [ ]:
embeddings_matrix

In [ ]:
np.save('recipes_bert_embeddings.npy', embeddings_matrix)

In [ ]:
loaded_embeddings = np.load('recipes_bert_embeddings.npy', allow_pickle=True)

# Embeddings de las reseñas

In [3]:
reviews = pd.read_parquet("data/foodcom-recipes-and-reviews/reviews.parquet")
len(reviews[reviews["Review"].isnull()])

0

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

In [6]:
embeddings = []
for review in tqdm(reviews['Review']):
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    review_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    embeddings.append(review_embedding)

  0%|          | 0/1401982 [00:00<?, ?it/s]c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 1401982/1401982 [4:19:52<00:00, 89.92it/s]  


In [7]:
embeddings_matrix = np.vstack(embeddings)

In [8]:
np.save('reviews_bert_embeddings.npy', embeddings_matrix)

In [ ]:
loaded_embeddings = np.load('reviews_bert_embeddings.npy', allow_pickle=True)